In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetracer.default_racecar import DefaultRacecar
from jetcam.utils import bgr8_to_jpeg

racecar = DefaultRacecar()

image_widget = ipywidgets.Image()

traitlets.dlink((racecar, 'image'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
display(image_widget)

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image

# define model
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_library_xy.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval()

# define preprocessing
mean = torch.Tensor([0.485, 0.456, 0.406]).float().cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).float().cuda()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).float()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
# control sliders
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.6, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.01, description='steering gain')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_bias_slider)

In [ ]:
# display sliders
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

In [ ]:
def execute(change):
    image = change['new']
    xy = model(preprocess(image)).detach().cpu().numpy().flatten()
    
    x_slider.value = float(xy[0])
    y_slider.value = 0.5 - float(xy[1])
    
    speed_slider.value = speed_gain_slider.value * y_slider.value
    steering_slider.value = x_slider.value * steering_gain_slider.value + steering_bias_slider.value
    
    racecar.steering = steering_slider.value
    racecar.throttle = speed_slider.value

In [ ]:
racecar.observe(execute, names='image')

In [ ]:
racecar.unobserve(execute, names='image')